In [35]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from art.attacks.extraction import CopycatCNN
from art.utils import load_dataset
from art.estimators.classification import TensorFlowV2Classifier

In [36]:
vulnerable_model = tf.keras.models.load_model("./results/model_no_defense.h5")

In [37]:
#Se utilizara una version del dataset del MNIST con 60k digitos a mano que incluye ART
(train_images, train_labels), (test_images, test_labels), min, max = load_dataset(name="mnist")

In [38]:
#Usamos las 10k observaciones que no fueron utilizadas en el modelo 
train_images_stolen = train_images[50000:]
train_labels_stolen = train_labels[50000:]

In [39]:
classifier_b = TensorFlowV2Classifier(
    model=vulnerable_model,
    nb_classes=10, #Numeros del 0 al 9
    input_shape=(28, 28, 1), #Es necesario conocer el formato de la data
    loss_object=tf.keras.losses.CategoricalCrossentropy(),
    train_step=None,  
    clip_values=(0.0, 1.0) #Valor de los pixeles normalizados (raw=[0,255])
)

In [40]:
# Creating the "neural net thief" object
# that will steal the original classifier
#Copycat usa inputs (incluso random) que provee al modelo y obtiene outputs, que usa para crear pares (input,output) y en base a eso entrena al modelo que roba al original
copycat_cnn = CopycatCNN(
    batch_size_fit=128,
    batch_size_query=128,
    nb_epochs=10,
    nb_stolen=20000, #Numero de consultas al model victima
    classifier=classifier_b
    )

In [41]:
#Definimos un nuevo modelo
def create_blank_model():
    # Defining the model
    model = tf.keras.models.Sequential([
        Conv2D(filters=32, kernel_size=2, activation="relu", input_shape=(28, 28, 1)),
        MaxPool2D(pool_size=2),
        Conv2D(filters=128, kernel_size=3, activation="relu"),
        MaxPool2D(pool_size=2),
        Conv2D(filters=32, kernel_size=2, activation="relu"),
        MaxPool2D(pool_size=2),
        Flatten(),        
        Dense(units=10, activation="softplus")
    ])

    # Compiling the model
    model.compile(
        optimizer="SGD",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
        )

    # Returning the model
    return model

In [42]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.CategoricalCrossentropy()

model_stolen = TensorFlowV2Classifier(
    model=create_blank_model(),
    nb_classes=10, #Numeros del 0 al 9
    input_shape=(28, 28, 1), #Es necesario conocer el formato de la data
    optimizer=optimizer,
    loss_object=loss,
    train_step=None,  
    clip_values=(0.0, 1.0) #Valor de los pixeles normalizados (raw=[0,255])
)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [43]:


# Extracting a thieved classifier
# by training the reference model
stolen_classifier = copycat_cnn.extract(
    x=train_images_stolen, 
    thieved_classifier=model_stolen
    )

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 16384, but received input with shape (128, 3136)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(128, 28, 28, 1), dtype=float32)
  • training=False
  • mask=None

In [44]:
# Testing the performance of the original classifier
score_original = classifier_b._model.evaluate(
    x=test_images, 
    y=test_labels
    )

# Testing the performance of the stolen classifier
score_stolen = stolen_classifier._model.evaluate(
    x=test_images, 
    y=test_labels
    )

# Comparing test losses
print(f"Original test loss: {score_original[0]:.2f} " 
      f"vs stolen test loss: {score_stolen[0]:.2f}")

# Comparing test accuracies
print(f"Original test accuracy: {score_original[1]:.2f} " 
      f"vs stolen test accuracy: {score_stolen[1]:.2f}")

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 16384, but received input with shape (None, 3136)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 28, 28, 1), dtype=float32)
  • training=False
  • mask=None